In [0]:
!pip install -q http://download.pytorch.org/whl/cu80/torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl torchvision
!pip install -q keras 

import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import numpy as np

import matplotlib.pyplot as plt
from IPython import display

# Путь джедая

## Введение в PyTorch

Вообще говоря, все приличные люди начинают изучение PyTorch с его внутренностей: всяких Tensor'ов, Variable'ов и прочих autograd'ов.

Обязательно почитайте статью, в которой подробно расписано это всё: [PyTorch — ваш новый фреймворк глубокого обучения (habrahabr)](https://habrahabr.ru/post/334380/).

А мы пока просто пробежимся по главному.

### Тензоры

Тензоры в PyTorch - это как numpy.array. Они есть разных типов, причем типизация строгая:
```python
torch.HalfTensor      # 16 бит, с плавающей точкой
torch.FloatTensor     # 32 бита,  с плавающей точкой
torch.DoubleTensor    # 64 бита, с плавающей точкой

torch.ShortTensor     # 16 бит, целочисленный, знаковый
torch.IntTensor       # 32 бита, целочисленный, знаковый
torch.LongTensor      # 64 бита, целочисленный, знаковый

torch.CharTensor      # 8 бит, целочисленный, знаковый
torch.ByteTensor      # 8 бит, целочисленный, беззнаковый
```

Например, можем создать тензор нужного размера:

In [0]:
x = torch.FloatTensor(3, 4)   # мусор
x.zero_()                     # нули

А можем сделать его из готового array'я:

In [0]:
x = np.arange(8).reshape(2, 4) + 5
y = np.arange(8).reshape(2, 4)
print(x)
print(y)

x = torch.LongTensor(x)
y = torch.LongTensor(y)
print(x)
print(y)

Можем делать стандартные операции:

In [0]:
print(x + y)

print(x * y)

print(x.type(torch.FloatTensor).log())

### Разминка
Возьмем простую математическую функцию с прикольным графиком:

$$ x(t) = t - 1.5 \cdot cos( 15 t) $$
$$ y(t) = t - 1.5 \cdot sin( 16 t) $$

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline

t = torch.linspace(-10, 10, steps = 10000)

# Посчитайте x(t), y(t)
x = <your code>
y = <your code>

plt.plot(x.numpy(), y.numpy())

## Automatic gradients

Всё это мог и numpy. А теперь переходим к тому, ради чего нужен PyTorch: backpropagation c помощью `Variable` и модуля `autograd`.

Начнём с простого примера с небольшим графом вычислений:

![graph](https://image.ibb.co/mWM0Lx/1_6o_Utr7_ENFHOK7_J4l_XJtw1g.png =500x)

In [0]:
x = autograd.Variable(torch.FloatTensor([-2]), requires_grad=True)
y = autograd.Variable(torch.FloatTensor([5]), requires_grad=True)
z = autograd.Variable(torch.FloatTensor([-4]), requires_grad=True)

q = x + y
f = q * z

f.backward()

In [0]:
print(x.grad)
print(y.grad)
print(z.grad)

__Нормальный пример:__ потренируем линейную регрессию на Boston Housing Dataset.

В целом, весь backpropagation выглядит как-то так:
1. У вас есть тензор. Он умеет в данные. Но вам не интересно просто в данные, хочется ещё и в градиенты.
2. Вы делаете ```a = autograd.Variable(data, requires_grad=True)```
3. Определяете функцию потерь `loss = whatever(a)`
4. Зовёте `loss.backward()`
5. ???
6. В ```a.grads``` записан нужный градиент.

Например:

In [0]:
from sklearn.datasets import load_boston
boston = load_boston()
plt.scatter(boston.data[:, -1], boston.target)

In [0]:
w = autograd.Variable(torch.zeros(1), requires_grad=True)
b = autograd.Variable(torch.zeros(1), requires_grad=True)

x = autograd.Variable(torch.FloatTensor(boston.data[:,-1] / 10))
y = autograd.Variable(torch.FloatTensor(boston.target))

In [0]:
y_pred = w * x + b
loss = torch.mean((y_pred - y)**2)

# Пересчитываем градиенты
loss.backward()

Теперь в поле `.grad` есть градиенты.

In [0]:
print("dL/dw = \n", w.grad)
print("dL/db = \n", b.grad)

Если несколько раз посчитать лосс, градиенты каждый раз будут прибавляться. Поэтому важно занулять градиенты между итерациями.

In [0]:
from IPython.display import clear_output

for i in range(100):
    y_pred = w * x  + b
    loss = torch.mean((y_pred - y)**2)
    loss.backward()

    w.data -= 0.05 * w.grad.data
    b.data -= 0.05 * b.grad.data
    
    # Зануляем градиенты
    w.grad.data.zero_()
    b.grad.data.zero_()
    
    if (i+1) % 5 == 0:
        clear_output(True)
        plt.scatter(x.data.numpy(), y.data.numpy())
        plt.scatter(x.data.numpy(), y_pred.data.numpy(), color='orange', linewidth=5)
        plt.show()

        print("loss = ", loss.data.numpy()[0])
        if loss.data.numpy()[0] < 0.5:
            print("Done!")
            break

### Cuda

Последняя особенность PyTorch - работа с cuda.

Вызовом `x = x.cuda()` мы перемещаем тензор на видеокарту. Точно так же перемещаются на видеокарту и все вычисления.

Кроме этого есть отдельный набор тензоров `torch.cuda.FloatTensor`, на случай, когда сразу понятно, что работаем на видеокарте.

Вернуться с видеокарты можно вызовом `.cpu()`.

---

А *путь джедая* только начинается. Прочитайте (всё ещё) статью [PyTorch — ваш новый фреймворк глубокого обучения](https://habrahabr.ru/post/334380/).

__Задание*__ Реализовать простую полносвязную сеть на чистом numpy.

# Дорожка простых смертных

Начнём уже решать задачу - а с тем, что такое PyTorch, разберемся подробнее по ходу дела :)

Вспомним про датасет с прошлого занятия.

In [0]:
from keras.datasets import imdb

NUM_WORDS = 10000

print('Loading data...')
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=NUM_WORDS)
print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

NUM_LABELS = len(np.unique(y_train))
print(NUM_LABELS, 'class classification')

print('Converting to bag-of-word matrix...')
def convert_to_bow(X):
  X_bow = np.zeros((len(X), NUM_WORDS))
  for i, review in enumerate(X):
    for ind in review:
      X_bow[i, ind] = 1
  return X_bow

X_train_bow, X_test_bow = convert_to_bow(X_train), convert_to_bow(X_test)

y_train, y_test = y_train.reshape((-1, 1)), y_test.reshape((-1, 1))

Мы тогда начали с BoW-модели. Повторим её на PyTorch!

*Напоминание*: на keras нам нужно было бы сделать следующее:
1. Определить модель, например:
```python 
model = Sequential()
model.add(Dense(1, activation='sigmoid', input_dim=NUM_WORDS))
```
2. Задать функцию потерь и оптимизатор:
```python
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
```

3. Запустить обучение:
```python
model.fit(X_train_bow, y_train, 
          batch_size=32,
          epochs=3,
          validation_data=(X_test_bow, y_test))
```

Конечно же, в PyTorch можно сделать всё то же самое, но гораздо сложнее :)

## Определение модели
Начнём с определения модели:

In [0]:
model = nn.Sequential()

model.add_module('layer', nn.Linear(NUM_WORDS, 1))

model = model.cuda()

print(model)

Пока всё просто, правда?

`Linear` вместо `Dense`, другие названия аргументов. И **нет** возможности задать функцию активации.

Ну, и добавился вызов `model.cuda()`

---

Дальше - сложнее.

Начнем с такой вот магии. О чём она - разберёмся чуть позже.

In [0]:
def get_batches(dataset, batch_size):
    X, Y = dataset
    n_samples = X.shape[0]

    indices = np.arange(n_samples)
    np.random.shuffle(indices)
    
    for start in range(0, n_samples, batch_size):
        end = min(start + batch_size, n_samples)
        
        batch_idx = indices[start:end]
    
        yield autograd.Variable(X[batch_idx, ]), autograd.Variable(Y[batch_idx, ])

X_train_bow, y_train = torch.cuda.FloatTensor(X_train_bow), torch.cuda.FloatTensor(y_train)
X_test_bow, y_test = torch.cuda.FloatTensor(X_test_bow), torch.cuda.FloatTensor(y_test)

Создадим свой собственный мини-батч:

In [0]:
X_batch, y_batch = next(get_batches((X_train_bow, y_train), 32))

Посмотрим на него глазами

In [0]:
X_batch, y_batch

Чтобы вычислить значение на этом батче, нужно позвать метод `forward`:

In [0]:
logit = model.forward(X_batch)

Смотрите, что он нам выдал:

In [0]:
logit

В вероятности сконвертировать эти значения можно с помощью `F.sigmoid`

In [0]:
<your code here>

## Функция потерь
Нам бы теперь как-нибудь настроить параметры. А каким образом это работало в keras?

Посмотрите на ту строчку с `model.compile`. Ту часть, где мы задавали `loss`.

Что, собственно, она означает? Она говорит, что функция потерь имеет такой вид:
$$ L = {1 \over N} \underset{X_i,y_i} \sum - [  y_i \cdot \log P(y_i | X_i) + (1-y_i) \cdot \log (1-P(y_i | X_i)) ]$$

А что предсказывает наша модель? Да просто какие-то значения. Если они большие - наверное, положительный класс. А если маленькие - отрицательный.

Можно было бы добавить в нашу модель ещё один слой:
```python
model.add_module('predictions_layer', nn.Sigmoid())
```
и тогда в ход пойдет `nn.BCELoss`, который делает ровно то, что написано в формуле сверху. 

А можно оставить всё как есть - но поставить `nn.BCEWithLogitsLoss`, который сам будет добавлять сигмоиду.

С помощью кого-то из них мы можем вычислять потери на нашем батче:


In [0]:
loss_function = nn.BCEWithLogitsLoss()

loss = loss_function(logit, y_batch)

Мы получили некоторое значение потерь:

In [0]:
loss

Но это не просто значение. У него есть `backward`!

In [0]:
loss.backward()

И мы можем получить градиент:

In [0]:
model.layer.weight.grad


## Оптимизатор

Теперь мы можем настраивать параметры.

Например, с помощью SGD:
$$\theta^{(t+1)} = \theta^{(t)} - \eta \nabla_\theta L(\theta)$$

Либо с помощью чего-то более сложного:

In [0]:
optimizer = optim.Adam(model.parameters())

print(model.layer.weight)

optimizer.step()

print(model.layer.weight)

Нужно вызывать `step()`, чтобы обновить параметры модели.

Последнее, но очень важное - **не забыть обнулить градиенты**. Делается это так:

In [0]:
optimizer.zero_grad()

## Цикл обучения

А теперь соберём всё вместе.

Напишем функцию-аналог `fit` в keras. 

Тут-то нам и понадобится `get_batches`. Keras, по доброте душевной, сам выделяет мини-батчи и вычисляет все потери, метрики (и не забывает обнулить градиенты).

In [0]:
loss_function = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters())

for epoch in range(10):
  train_epoch_loss = 0
  train_epochs_count = 0
  train_correct_count = 0
  for X_batch, y_batch in get_batches((X_train_bow, y_train), 128):
    # 1. Обнуляем градиенты
    optimizer.zero_grad()
    
    # 2. Запускаем forward-проход
    logits = <your code>
    
    # 3. Вычисляем потери
    loss = <your code here too>
    
    # 4. Вычисляем градиенты на backward-проходе. А какие, собственно, градиенты он подсчитает?
    <and here, please>
    
    # 5. Оптимизируем параметры. Откуда он знает, какие параметры оптимизировать?
    <the last piece of code>
    
    # Агрегируем лосс для вывода
    train_epoch_loss += loss.data.cpu()[0]
    train_epochs_count += 1
    
    # Вычисляем accuracy
    probs = F.sigmoid(logits)
    predictions = (probs > 0.5).type(torch.cuda.LongTensor)
    train_correct_count += np.sum((predictions == y_batch.type(torch.cuda.LongTensor)).cpu().data.numpy())
  
  val_epoch_loss = 0
  val_epochs_count = 0
  val_correct_count = 0
  <your turn, calculate validation loss and accuracy>
  
  print('Train Loss = {:.5f}, Train Accuracy = {:.2%}, Val Loss = {:.5f}, Val Accuracy = {:.2%}'.format(
          train_epoch_loss / train_epochs_count, train_correct_count / len(y_train), 
          val_epoch_loss / val_epochs_count, val_correct_count / len(y_test)))

**Задание** Написать собственный оптимизатор (например, SGD) вместо Adam.

Параметры модели можно получить так:

In [0]:
for param in model.parameters():
  print(param)

У них есть `.grad`, который можно использовать для обновления параметров.

При этом занулять градиенты придется так: `model.zero_grad()`

## Module API

Кроме `Sequential` есть и более функциональный способ задавать модели:

In [0]:
class BoWClassifier(nn.Module):  # Наследуемся от nn.Module
    def __init__(self, num_labels, vocab_size):
        super().__init__()

        # Определяем слои, которые будем использовать
        self.linear = nn.Linear(vocab_size, num_labels)

    def forward(self, bow_vec):
        # Пропускаем вектор через них
        return self.linear(bow_vec)

# Word2Vec

Попробуем потренировать собственные эмбеддинги.

Сначала скачаем датасет (следующие четыре ячейки нужно просто запустить и надеяться, что в них всё правильно)

In [0]:
!wget -O text8.zip http://mattmahoney.net/dc/text8.zip
!unzip text8.zip

In [0]:
with open('text8') as f:
  words = f.read().split()
print("data_size = {0}".format(len(words)))

In [0]:
# Only N = 50000 the most frequent words is considered
# The other marked with token `UNK` (unknown)

from collections import Counter

def build_dataset(words, vocabulary_size):
  count = [[ "UNK", -1 ]]
  count.extend(Counter(words).most_common(vocabulary_size-1))
  print("Least frequent word: ", count[-1])
  word_to_index = { word: i for i, (word, _) in enumerate(count) }
  data = [word_to_index.get(word, 0) for word in words] # map unknown words to 0
  unk_count = data.count(0) # Number of unknown words
  count[0][1] = unk_count
  index_to_word= dict(zip(word_to_index.values(), word_to_index.keys()))
  
  return data, count, word_to_index, index_to_word

vocabulary_size = 50000
data, count, word_to_index, index_to_word = build_dataset(words, vocabulary_size)

# Everything you need to know about the dataset

print("data: {0}".format(data[:10]))
print("count: {0}".format(count[:10]))
print("index_to_word: {0}".format(list(index_to_word.items())[:10]))

In [0]:
from collections import deque

def generate_batch(data_index, data_size, batch_size, bag_window):
  span = 2 * bag_window + 1 # [ bag_window, target, bag_window ]
  batch = np.ndarray(shape = (batch_size, span - 1), dtype = np.int32)
  labels = np.ndarray(shape = (batch_size), dtype = np.int32)
  
  data_buffer = deque(maxlen = span)
  
  for _ in range(span):
    data_buffer.append(data[data_index])
    data_index = (data_index + 1) % data_size
    
  for i in range(batch_size):
    data_list = list(data_buffer)
    labels[i] = data_list.pop(bag_window)
    batch[i] = data_list
    
    data_buffer.append(data[data_index])
    data_index = (data_index + 1) % data_size
  return data_index, batch, labels


print("data = {0}".format([index_to_word[each] for each in data[:16]]))
data_index, data_size, batch_size = 0, len(data), 4
for bag_window in [1, 2]:
  data_index, batch, labels = generate_batch(data_index, data_size, batch_size, bag_window)
  print("bag_window = {0}".format(bag_window))
  print("batch = {0}".format([[index_to_word[index] for index in each] for each in batch]))
  print("labels = {0}\n".format([index_to_word[each] for each in labels.reshape(4)]))

Напомню, мы тут собрались научиться получать эмбеддинги слов. Ну, и получать такие прикольные штуки, если повезёт:
![Word vectors relations](https://www.tensorflow.org/images/linear-relationships.png =700x)

Вообще говоря, каждое слово можно представлять и просто как индекс в словаре: 
$$\overbrace{\left[ 0, 0, \dots, 1, \dots, 0, 0 \right]}^\text{|V| elements}$$

Основной недостаток - размер таких векторов и отсутствие интерпретируемости. Хочется, чтобы было как-то так:
$$q_\text{mathematician} = \left[ \overbrace{2.3}^\text{can run},
\overbrace{9.4}^\text{likes coffee}, \overbrace{-5.5}^\text{majored in Physics}, \dots \right]$$
$$q_\text{physicist} = \left[ \overbrace{2.5}^\text{can run},
\overbrace{9.1}^\text{likes coffee}, \overbrace{6.4}^\text{majored in Physics}, \dots \right]$$

По таким векторам уже можно считать похожесть:
$$\text{Similarity}(\text{physicist}, \text{mathematician}) = \frac{q_\text{physicist} \cdot q_\text{mathematician}}
{\| q_\text{physicist} \| \| q_\text{mathematician} \|} = \cos (\phi)$$

Для тренировки таких представлений есть два наиболее популярных способа: skip-gram и CBoW.

## Continuous Bag of Words (CBoW)
В этой модели мы учимся предсказывать слово по его контексту:

![CBoW](http://evelinag.com/fsharpexchange2017/images/word2vec-cbow.png =500x)

Таким образом, учимся моделировать $P(w_c| w_{c-k}, \ldots, w_{c-1}, w_{c+1}, \ldots, w_{c+k})$.

При этом тренируются две матрицы. Собственно, матрица эмбеддингов $W_1$ и матрица выходного слоя $W_2$.

Вообще говоря, можно особо не думать и cчитать $softmax(W_2 q_c + b)$, где $q_c$ - сумма эмбеддингов слов из контекста, т.е. $q_c = \sum_{w_i \in c} W_1 w_i$. 

В результате получим некоторое распределение вероятностей на словаре, а нам нужно просто оптимизировать наши матрицы так, чтобы вероятность нужного слова была максимальной. 

Оптимизируется тут, как всегда, кросс-энтропия.

Разберемся чуть подробнее. Пусть $\tilde w_c$ -- это предсказываемое слово. $w_{c-k}, \ldots, w_{c-1}, w_{c+1}, \ldots, w_{c+k}$ - его контекст.

Сначала мы считаем эмбеддинги этих контекстных слов: $u_j = W_1 w_j, \ j = c-k, \ldots, c+k, \ j \neq c$.

Потом мы суммируем эти эмбеддинги: $u_c = \sum_j u_j$. Получили как бы эмбеддинг контекста.

А теперь вспоминаем про матрицу $W_2$. Она тоже содержит "эмбеддинги" векторов: $i$-тый столбец $v_i$ - эмбеддинг $i$-того слова.

Вот давайте учиться тому, чтобы эмбеддинг контекста был максимально похож на эмбеддинг слова:
$$-\log \frac{\exp(v_c^T u_c)}{\sum_{i=1}^{|V|} \exp(v_i^T u_c)} \to \min.$$

Заметьте, мы считаем скалярное произведение между парой векторов, почти как в `Similarity` выше

*Ничего не понятно, да? :(*

In [0]:
class CBOW(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super().__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.out_layer = nn.Linear(embedding_dim, vocab_size)

    def forward(self, inputs):
        # Просто эмбеддим все входные слова и суммируем полученные эмбеддинги = u_c
        embeds = self.embeddings(inputs).sum(dim=1)
        # Вычисляем W_2 x u_c = {v_i^T x u_c} из формулы выше
        out = self.out_layer(embeds)
        # Считаем log_softmax - логарифмы вероятностей того, что такое слово - центральное
        return F.log_softmax(out, dim=1)
      
# Строим эмбеддинги размерности 50
model = CBOW(vocabulary_size, 50)
model = model.cuda()

# NLLLoss - кросс-энтропийная функция потерь для логарифмов вероятностей
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)  

In [0]:
data_index, data_size, batch_size, bag_window = 0, len(data), 64, 2
  
loss_every_nsteps = 1000
total_loss = 0
for step in range(50000):
    data_index, batch, labels = generate_batch(data_index, data_size, batch_size, bag_window)
    batch, labels = autograd.Variable(torch.cuda.LongTensor(batch)), autograd.Variable(torch.cuda.LongTensor(labels))

    optimizer.zero_grad()

    log_probs = model(batch)

    loss = loss_function(log_probs, labels)
    loss.backward()

    optimizer.step()

    total_loss += loss.data
    if step % loss_every_nsteps == 0:
        if step > 0:
            total_loss /= loss_every_nsteps
            display.clear_output(True)
            print("step = {0}, average_loss = {1}".format(step, total_loss.cpu().numpy()[0]))
            total_loss = 0

Визуализировать всё это можно так:

In [0]:
from sklearn.manifold import TSNE

num_points = 250

tsne = TSNE(perplexity=10, n_components=2, init="pca", n_iter=5000)
two_d_embeddings = tsne.fit_transform(model.embeddings.weight.data.cpu().numpy()[1:num_points+1, :])

plt.figure(figsize=(15,15))
words = [index_to_word[i] for i in range(1, num_points+1)]

for i, label in enumerate(words):
    x, y = two_d_embeddings[i, :]
    plt.scatter(x, y)
    plt.annotate(label, xy=(x, y), xytext=(5, 2), textcoords="offset points",
                   ha="right", va="bottom")
plt.show()

## Negative Sampling
Вообще говоря, считать softmax на большом словаре - очень долго и вычислительно сложно.

Один из способов справиться с этим - использовать *Negative Sampling*.

По сути, вместо предсказания индекса слова по контексту предсказывается вероятность того, что такое слово может быть в таком контексте: $P(D=1|w,c)$.

Можно использовать обычную сигмоиду для получения данной вероятности: 
$$P(D=1|w, c) = \sigma(v_w^T u_c) = \frac 1 {1 + \exp(-v^T_w u_c)}.$$

Процесс обучения тогда выглядит так: для каждой пары слово и его контекст генерируем набор отрицательных примеров:

![Negative Sampling](https://image.ibb.co/dnOUDH/Negative_Sampling.png =350x)

Для CBoW функция потерь будет выглядеть так:
$$-\log \sigma(v_c^T u_c) - \sum_{k=1}^K \log \sigma(-\tilde v_k^T u_c),$$
где $\tilde v_1, \ldots, \tilde v_K$ - сэмплированные негативные примеры.

Сравните эту формулу с обычным CBoW:
$$-v_c^T u_c + \log \sum_{i=1}^{|V|} \exp(v_i^T u_c).$$

Обычно слова сэмплируются из $U^{3/4}$, где $U$ - униграмное распределение, т.е частоты появления слова делённые на суммарое число слов. 

Частотности мы уже считали: они получаются в `Counter(words)`. Достаточно просто преобразовать их в вероятности и домножить эти вероятности на $\frac 3 4$. Почему $\frac 3 4$? Некоторую интуицию можно найти в следующем примере:

$$P(\text{is}) = 0.9, \ P(\text{is})^{3/4} = 0.92$$
$$P(\text{Constitution}) = 0.09, \ P(\text{Constitution})^{3/4} = 0.16$$
$$P(\text{bombastic}) = 0.01, \ P(\text{bombastic})^{3/4} = 0.032$$

Вероятность для высокочастотных слов особо не увеличилась (относительно), зато низкочастотные будут выпадать с заметно большей вероятностей.

**Задание** Реализуйте свой Negative Sampling.

In [0]:
class NegativeSamplingCBOW(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super().__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.out_embeddings = nn.Embedding(vocab_size, embedding_dim)

    def forward(self, inputs, targets, num_samples):
        '''
        inputs : (batch_size, context_size)
        targets: (batch_size)
        '''
        # Находим u_c
        embeds = self.embeddings(inputs).sum(dim=1)
        # Считаем v_c
        outputs = self.out_embeddings(targets)
        
        <Sample indices for v_1, ..., v_K>
        <Calculate v_1, ..., v_K using self.out_embeddings>
        
        <Calculate loss>
        return loss
        
# Строим эмбеддинги размерности 50
model = CBOW(vocabulary_size, 50)
model = model.cuda()

# Обратите внимание, loss уже не нужен!
# Считаем его прямо в классе
# Таким образом, просто делаем loss = model(batch)
optimizer = optim.SGD(model.parameters(), lr=0.01)  

## Skip-Gram

В Skip-gram модели всё наоборот. Предсказываются слова из контекста по данному слову.

![Skip-gram](https://adriancolyer.files.wordpress.com/2016/04/word2vec-skip-gram.png?w=600)

Теперь учимся моделировать вероятность $P(w_{c-k}, \ldots, w_{c-1}, w_{c+1}, \ldots, w_{c+k} | w_c)$. Это, конечно, сложно, поэтому упростим всё - используем наивного Байеса:
$P(w_{c-k}, \ldots, w_{c-1}, w_{c+1}, \ldots, w_{c+k} | w_c) = \prod_{j=-k, j \neq 0}^k P(w_{c+j} | w_c)$. Посмотрите ещё раз на картинку - там наприсовано именно это (точнее, там нарисовано, что тренируется всего одна матрица $W_2$).

Оптимизировать нужно всё ту же кросс-энтропию. Только теперь уже нужно суммировать кросс-энтропийные потери для всех слов в контексте.

**Задание** Реализовать, ага.

Начнем с генерации батча. Представить, что там происходит, проще всего по этой картинке:
![skip-gram-batch](https://raw.githubusercontent.com/deepmipt/deep-nlp-seminars/44f574efe79ca1613d77336e4163061f9d5566c6/seminar_02/pics/training_data.png =600x)

Т.е. тренировочные данные состоят из пар (слово, слово из контекста).

В результате, если вся выборка будет состоять из одного этого предложения, распределение вероятностей, которое мы стремимся получить, для слова `brown` будет таким:

![skip-gram-illustration](https://image.ibb.co/eJ3otH/SkipGram.png =600x)

Каждое из слов в его контексте получает одинаковую вероятность $1 \over 4$, а все остальные слова - ноль. 

In [0]:
import random

def generate_batch_2(data_index, data_size, batch_size, num_skips, skip_window):
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    
    batch = np.ndarray(shape = batch_size, dtype = np.int32)
    labels = np.ndarray(shape = (batch_size, 1), dtype = np.int32)
    span = 2 * skip_window + 1
    data_buffer = deque(maxlen = span)
    for _ in range(span):
        data_buffer.append(data[data_index])
        data_index = (data_index + 1) % data_size
    
    for i in range(batch_size // num_skips):
        target, targets_to_avoid = skip_window, [skip_window]
        for j in range(num_skips):
            while target in targets_to_avoid: 
                target = random.randint(0, span - 1)
            targets_to_avoid.append(target)
            batch[i * num_skips + j] = data_buffer[skip_window]
            labels[i * num_skips + j, 0] = data_buffer[target]
        data_buffer.append(data[data_index])
        data_index = (data_index + 1) % data_size
    return data_index, batch, labels


print("data = {0}\n".format([index_to_word[each] for each in data[:32]]))
data_index, data_size = 0, len(data)
for num_skips, skip_window in [(2, 1), (4, 2)]:
    data_index = 0
    data_index, batch, labels = generate_batch_2(data_index=data_index, 
                                               data_size=data_size, 
                                               batch_size=16, 
                                               num_skips=num_skips, 
                                               skip_window=skip_window)
    print("data_index = {0}, num_skips = {1}, skip_window = {2}".format( data_index, num_skips, skip_window))
    print("batch = {0}".format([index_to_word[each] for each in batch]))
    print("labels = {0}\n".format([index_to_word[each] for each in labels.reshape(16)]))

In [0]:
class SkipGram(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super().__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.out_layer = nn.Linear(embedding_dim, vocab_size)

    def forward(self, inputs):
        <your code here>      